# **Data Exploration**

In this notebook we'll look at one of the first elements involved in any data engineering project - getting to know what the inputs might look like.

This notebook was adapted from a Netflix data engineering workshop which you can find here: https://github.com/NFLX-WIBD/WIBD-Workshops-2018.


### Running Pyspark in Colab

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab. Follow the steps to install the dependencies:

In [0]:
%%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop3.2.tgz
!tar -xvf spark-3.0.0-preview2-bin-hadoop3.2.tgz  
!pip install -q findspark

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop3.2"

Run a local spark session to test your installation:

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Congrats! Your Colab is ready to run Pyspark. Let's explore some data from the 

In [0]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/WDICountry.csv     
  inflating: __MACOSX/data/._WDICountry.csv  
  inflating: data/WDIData.csv        
  inflating: __MACOSX/data/._WDIData.csv  
  inflating: data/WDISeries.csv      
  inflating: __MACOSX/data/._WDISeries.csv  


### Step 1 - What inputs do you have?

In [0]:
%%bash
ls ../data

ls: cannot access '../data': No such file or directory


### Step 2 - Get your tools setup

As part of this we will be using [PySpark](http://spark.apache.org/docs/2.1.1/api/python/index.html) to inspect the data on hand and also gather some basic details.

In [0]:
import pandas as pd

from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  20, truncate = True):
    if(truncate):
        pd.set_option('display.max_colwidth', 50)
    else:
        pd.set_option('display.max_colwidth', -1)
    pd.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pd.reset_option('display.max_rows')

In [0]:
#Creating a spark session
spark = SparkSession.builder.appName("Data Exploration").getOrCreate()

### Step 3 - Look inside your data

We need to look at how our data is composed:
1. Format
2. Structure
3. Size
4. Dimensions

In this example our input is a CSV file with a header.  Let's try to see what the data looks like

#### Read The Data

In [0]:
#Read the file into a Spark Data Frame
country = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("data/WDICountry.csv")

#### Inspect the schema of the file you just read

In [0]:
country.printSchema()

root
 |-- Country Code: string (nullable = true)
 |-- Short Name: string (nullable = true)
 |-- Table Name: string (nullable = true)
 |-- Long Name: string (nullable = true)
 |-- 2-alpha code: string (nullable = true)
 |-- Currency Unit: string (nullable = true)
 |-- Special Notes: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Income Group: string (nullable = true)
 |-- WB-2 code: string (nullable = true)
 |-- National accounts base year: string (nullable = true)
 |-- National accounts reference year: integer (nullable = true)
 |-- SNA price valuation: string (nullable = true)
 |-- Lending category: string (nullable = true)
 |-- Other groups: string (nullable = true)
 |-- System of National Accounts: string (nullable = true)
 |-- Alternative conversion factor: string (nullable = true)
 |-- PPP survey year: string (nullable = true)
 |-- Balance of Payments Manual in use: string (nullable = true)
 |-- External debt Reporting status: string (nullable = true)
 |-- Sys

#### Take a look at some sample data

You can run <dataframe>.show() to look at the sample data.  However the output is not well formatted so we will use our helper function to look at the data.

In [0]:
showDF(country, truncate = False)

,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code,National accounts base year,National accounts reference year,SNA price valuation,Lending category,Other groups,System of National Accounts,Alternative conversion factor,PPP survey year,Balance of Payments Manual in use,External debt Reporting status,System of trade,Government Accounting concept,IMF data dissemination standard,Latest population census,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data
0,ABW,Aruba,Aruba,Aruba,AW,Aruban florin,SNA data for 2000-2011 are updated from official government statistics; 1994-1999 from UN databases. Base year has changed from 1995 to 2000.,Latin America & Caribbean,High income,AW,2000,NaN,Value added at basic prices (VAB),None,None,Country uses the 1993 System of National Accounts methodology,None,2011,BPM5 (Converted into BPM6 by IMF),None,General trade system,None,Enhanced General Data Dissemination System (e-GDDS),2010,None,None,Yes,None,NaN,2016.0,NaN
1,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,"Fiscal year end: March 20; reporting period for national accounts data is calendar year, estimated to insure consistency between national accounts and fiscal data. National accounts data are sourced from the IMF and differ from the Central Statistics Organization numbers due to exclusion of the opium economy.",South Asia,Low income,AF,2002/03,NaN,Value added at basic prices (VAB),IDA,HIPC,Country uses the 1993 System of National Accounts methodology,None,None,BPM6,Actual,General trade system,Consolidated central government,Enhanced General Data Dissemination System (e-GDDS),1979,"Demographic and Health Survey, 2015","Integrated household survey (IHS), 2011",None,None,NaN,2016.0,2000.0
2,AGO,Angola,Angola,People's Republic of Angola,AO,Angolan kwanza,None,Sub-Saharan Africa,Lower middle income,AO,2002,NaN,Value added at producer prices (VAP),IBRD,None,Country uses the 1993 System of National Accounts methodology,1991–96,2011,BPM6,Actual,Special trade system,Budgetary central government,Enhanced General Data Dissemination System (e-GDDS),2014,"Demographic and Health Survey, 2015/16","Integrated household survey (IHS), 2008/09",None,None,NaN,2016.0,2005.0
3,ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,None,Europe & Central Asia,Upper middle income,AL,Original chained constant price data are rescaled.,1996.0,Value added at basic prices (VAB),IBRD,None,Country uses the 2008 System of National Accounts methodology,None,Rolling,BPM6,Actual,Special trade system,Consolidated central government,Enhanced General Data Dissemination System (e-GDDS),2011,"Demographic and Health Survey, 2008/09","Living Standards Measurement Study Survey (LSMS), 2012",Yes,2012,2013.0,2016.0,2006.0
4,AND,Andorra,Andorra,Principality of Andorra,AD,Euro,WB-3 code changed from ADO to AND to align with ISO code.,Europe & Central Asia,High income,AD,2000,NaN,Value added at basic prices (VAB),None,None,Country uses the 1993 System of National Accounts methodology,None,None,None,None,General trade system,None,None,2011. Population data compiled from administrative registers.,None,None,Yes,None,NaN,NaN,NaN
5,ARB,Arab World,Arab World,Arab World,1A,None,Arab World aggregate. Arab World is composed of members of the League of Arab States.,None,None,1A,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,2016.0,NaN
6,ARE,United Arab Emirates,United Arab Emirates,United Arab Emirates,AE,U.A.E. dirham,None,Middle East & North Africa,High income,AE,2010,NaN,Value added at basic prices (VAB),None,None,Country uses the 1993 System of National Accounts methodology,None,2011,None,None,Special trade system,Consolidated central government,Enhanced General Data Dissemination System (e-GDDS),2010,"World

#### Get Some Basic Stats

In [0]:
#Count the number of records in the dataframe
country.count()

263

#### Examining Dimensions
##### How many different regions do the various countries belong to ?

In [0]:
showDF(country.select('Region').distinct(), truncate = False)

,Region
0,Latin America & Caribbean
1,South Asia
2,Sub-Saharan Africa
3,Europe & Central Asia
4,None
5,Middle East & North Africa
6,East Asia & Pacific
7,North America


##### How many different income groups do we have across countries?

In [0]:
showDF(country.select('Income Group').distinct(), truncate = False)

,Income Group
0,High income
1,Low income
2,Lower middle income
3,Upper middle income
4,None


#### By applying the same steps as we did for the "WDICountry.csv" dataset, we can see what the rest of the datasets look like

###### WDISeries.csv

In [0]:
series = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("data/WDISeries.csv")

In [0]:
series.printSchema()

root
 |-- Series Code: string (nullable = true)
 |-- Topic: string (nullable = true)
 |-- Indicator Name: string (nullable = true)
 |-- Short definition: string (nullable = true)
 |-- Long definition: string (nullable = true)
 |-- Unit of measure: string (nullable = true)
 |-- Periodicity: string (nullable = true)
 |-- Base Period: string (nullable = true)
 |-- Other notes: string (nullable = true)
 |-- Aggregation method: string (nullable = true)
 |-- Limitations and exceptions: string (nullable = true)
 |-- Notes from original source: string (nullable = true)
 |-- General comments: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Statistical concept and methodology: string (nullable = true)
 |-- Development relevance: string (nullable = true)
 |-- Related source links: string (nullable = true)
 |-- Other web links: string (nullable = true)
 |-- Related indicators: string (nullable = true)
 |-- License Type: string (nullable = true)
 |-- _c20: string (nullable = tru

In [0]:
showDF(series)

,Series Code,Topic,Indicator Name,Short definition,Long definition,Unit of measure,Periodicity,Base Period,Other notes,Aggregation method,Limitations and exceptions,Notes from original source,General comments,Source,Statistical concept and methodology,Development relevance,Related source links,Other web links,Related indicators,License Type,_c20
0,AG.AGR.TRAC.NO,Environment: Agricultural production,"Agricultural machinery, tractors",None,Agricultural machinery refers to the number of...,None,Annual,None,None,Sum,The data are collected by the Food and Agricul...,None,None,"Food and Agriculture Organization, electronic ...",A tractor provides the power and traction to m...,Agricultural land covers more than one-third o...,None,None,None,CC BY-4.0,None
1,AG.CON.FERT.PT.ZS,Environment: Agricultural production,Fertilizer consumption (% of fertilizer produc...,None,Fertilizer consumption measures the quantity o...,None,Annual,None,None,Weighted average,The FAO has revised the time series for fertil...,None,None,"Food and Agriculture Organization, electronic ...",Fertilizer consumption measures the quantity o...,"Factors such as the green revolution, has led ...",None,None,None,CC BY-4.0,None
2,AG.CON.FERT.ZS,Environment: Agricultural production,Fertilizer consumption (kilograms per hectare ...,None,Fertilizer consumption measures the quantity o...,None,Annual,None,None,Weighted average,The FAO has revised the time series for fertil...,None,None,"Food and Agriculture Organization, electronic ...",Fertilizer consumption measures the quantity o...,"Factors such as the green revolution, has led ...",None,None,None,CC BY-4.0,None
3,AG.LND.AGRI.K2,Environment: Land use,Agricultural land (sq. km),None,Agricultural land refers to the share of land ...,None,Annual,None,None,Sum,The data are collected by the Food and Agricul...,None,None,"Food and Agriculture Organization, electronic ...",Agricultural land constitutes only a part of a...,Agricultural land covers more than one-third o...,None,None,None,CC BY-4.0,None
4,AG.LND.AGRI.ZS,Environment: Land use,Agricultural land (% of land area),None,Agricultural land refers to the share of land ...,None,Annual,None,None,Weighted average,The data are collected by the Food and Agricul...,None,None,"Food and Agriculture Organization, electronic ...",Agriculture is still a major sector in many ec...,Agricultural land covers more than one-third o...,None,None,None,CC BY-4.0,None
5,AG.LND.ARBL.HA,Environment: Land use,Arable land (hectares),None,Arable land (in hectares) includes land define...,None,Annual,None,None,None,The Food and Agriculture Organization (FAO) tr...,None,None,"Food and Agriculture Organization, electronic ...",Temporary fallow land refers to land left fall...,Agricultural land covers more than one-third o...,None,None,None,CC BY-4.0,None
6,AG.LND.ARBL.HA.PC,Environment: Land use,Arable land (hectares per person),None,Arable land (hectares per person) includes lan...,None,Annual,None,None,Weighted Average,The Food and Agriculture Organization (FAO) tr...,None,None,"Food and Agriculture Organization, electronic ...",Temporary fallow land refers to land left fall...,Agricultural land covers about one-third of th...,None,None,None,CC BY-4.0,None
7,AG.LND.ARBL.ZS,Environment: Land use,Arable land (% of land area),None,Arable land includes land defined by the FAO a...,None,Annual,None,None,Weighted average,The Food and Agriculture Organization (FAO) tr...,None,None,"Food and Agriculture Organization, electronic ...",Temporary fallow land refers to land left fall...,Agricultural land covers more than one-third o...,None,None,None,CC BY-4.0,None
8,AG.LND.CREL.HA,Environment: Agricultural production,Land under cereal production (hectares),None,Land under cereal production refers to harvest...,None,Annual,None,None,Sum,The data are collected by the Food and Agricul...,None,None,"Food and Agriculture Organization, electronic ...","Cereals production includes wheat, rice, maize...",The cultivati

In [0]:
series.count()

1593

#### Examining Dimensions
##### What are the different periodicities or aggregation methods we might expect to see in the data ?

In [0]:
showDF(series.select('Periodicity').distinct(), truncate = False)

,Periodicity
0,Annual
1,None
2,"International Civil Aviation Organization, Civil Aviation Statistics of the World and ICAO staff estimates."
3,Quarterly (represented as Annual)


In [0]:
showDF(series.select('Aggregation Method').distinct(), truncate = False)

,Aggregation Method
0,Sum
1,Weighted average
2,None
3,Weighted Average
4,Gap-filled total
5,Median
6,Unweighted average
7,Simple average
8,Linear mixed-effect model estimates


## Exercise

Repeat the same steps for the `WDIData.csv` file and read it into a dataframe called `indicators`.

In [0]:
# Read the data
indicators = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("data/WDIData.csv")

indicators.count()

420024

In [0]:
# Inspect the schema


In [0]:
# Look at sample records


In [0]:
# Get some basic stats


# Data Transformation and Quality

### Step 4 - Transform the data

#### Transform the Country Dataset
- Select the columns that we will need for our data model
- Rename columns for data ingestion

We'll be using the various operations supported for a DataFrame.  You can view the complete list [here](http://spark.apache.org/docs/2.1.1/api/python/pyspark.sql.html#pyspark.sql.DataFrame) .

In [0]:
countryDim = country \
    .select("2-alpha code", "Country Code", "Short Name", "Long Name", "Region", "Income Group") \
    .withColumnRenamed("2-alpha code", "country_iso_code") \
    .withColumnRenamed("Country Code", "wb_country_code") \
    .withColumnRenamed("Short Name", "country_name") \
    .withColumnRenamed("Long Name", "country_long_name") \
    .withColumnRenamed("Region", "region") \
    .withColumnRenamed("Income Group", "income_group")
    
showDF(countryDim)

,country_iso_code,wb_country_code,country_name,country_long_name,region,income_group
0,AW,ABW,Aruba,Aruba,Latin America & Caribbean,High income
1,AF,AFG,Afghanistan,Islamic State of Afghanistan,South Asia,Low income
2,AO,AGO,Angola,People's Republic of Angola,Sub-Saharan Africa,Lower middle income
3,AL,ALB,Albania,Republic of Albania,Europe & Central Asia,Upper middle income
4,AD,AND,Andorra,Principality of Andorra,Europe & Central Asia,High income
5,1A,ARB,Arab World,Arab World,None,None
6,AE,ARE,United Arab Emirates,United Arab Emirates,Middle East & North Africa,High income
7,AR,ARG,Argentina,Argentine Republic,None,None
8,AM,ARM,Armenia,Republic of Armenia,Europe & Central Asia,Lower middle income
9,AS,ASM,American Samoa,American Samoa,East Asia & Pacific,Upper middle income


#### You can also do similar transformations in sql
Let's rename the country_name and country_long_name columns

In [0]:
#Lets you create a view that you can use in SQL queries
countryDim.createOrReplaceTempView("country")

In [0]:
transformQuery = """
select 
    country_iso_code,
    wb_country_code,
    country_name as name,
    country_long_name as long_name,
    region,
    income_group
from 
    country
"""

showDF(spark.sql(transformQuery))

,country_iso_code,wb_country_code,name,long_name,region,income_group
0,AW,ABW,Aruba,Aruba,Latin America & Caribbean,High income
1,AF,AFG,Afghanistan,Islamic State of Afghanistan,South Asia,Low income
2,AO,AGO,Angola,People's Republic of Angola,Sub-Saharan Africa,Lower middle income
3,AL,ALB,Albania,Republic of Albania,Europe & Central Asia,Upper middle income
4,AD,AND,Andorra,Principality of Andorra,Europe & Central Asia,High income
5,1A,ARB,Arab World,Arab World,None,None
6,AE,ARE,United Arab Emirates,United Arab Emirates,Middle East & North Africa,High income
7,AR,ARG,Argentina,Argentine Republic,None,None
8,AM,ARM,Armenia,Republic of Armenia,Europe & Central Asia,Lower middle income
9,AS,ASM,American Samoa,American Samoa,East Asia & Pacific,Upper middle income


### Step 5 - Check the data quality

##### Do all countries have 2 character country_iso_codes ?

We will use some sql functions supported in PySpark for this exercise.  You can find a complete list of functions supported [here](http://spark.apache.org/docs/2.1.1/api/python/pyspark.sql.html#module-pyspark.sql.functions) .

In [0]:
countryCodeLength = countryDim.select(length("country_iso_code").alias("column_length")) \
    .groupBy("column_length") \
    .agg(count("*").alias("cnt")) \
    .filter("cnt > 1") 
showDF(countryCodeLength)

,column_length,cnt
0,2,262


In [0]:
#You can also do this in SQL
countryCodeLengthQuery = """
select 
    length(country_iso_code) as column_length,
    count(1) as cnt
from 
    country
group by 
    length(country_iso_code)
having 
    count(1) > 1
"""

showDF(spark.sql(countryCodeLengthQuery))

,column_length,cnt
0,2,262


## Quick Exercise
Do you notice a difference in the counts returned by the Quality check for the 2-character country_iso_code vs the original row count for the WDICountry.csv ?

In [0]:
# Paste your solution here


##### Do we have duplicate records for any of the key columns ?

In [0]:
showDF(countryDim.groupBy("country_iso_code").agg(count("*").alias("cnt")).filter("cnt > 1"))

showDF(countryDim.groupBy("wb_country_code").agg(count("*").alias("cnt")).filter("cnt > 1"))

showDF(countryDim.groupBy("country_name").agg(count("*").alias("cnt")).filter("cnt > 1"))

,country_iso_code,cnt


,wb_country_code,cnt


,country_name,cnt


### Step 6 - Fix any issues with data quality

In [0]:
countryDimFinal = countryDim.filter("country_iso_code is not null")

showDF(countryDimFinal)

,country_iso_code,wb_country_code,country_name,country_long_name,region,income_group
0,AW,ABW,Aruba,Aruba,Latin America & Caribbean,High income
1,AF,AFG,Afghanistan,Islamic State of Afghanistan,South Asia,Low income
2,AO,AGO,Angola,People's Republic of Angola,Sub-Saharan Africa,Lower middle income
3,AL,ALB,Albania,Republic of Albania,Europe & Central Asia,Upper middle income
4,AD,AND,Andorra,Principality of Andorra,Europe & Central Asia,High income
5,1A,ARB,Arab World,Arab World,None,None
6,AE,ARE,United Arab Emirates,United Arab Emirates,Middle East & North Africa,High income
7,AR,ARG,Argentina,Argentine Republic,None,None
8,AM,ARM,Armenia,Republic of Armenia,Europe & Central Asia,Lower middle income
9,AS,ASM,American Samoa,American Samoa,East Asia & Pacific,Upper middle income


In [0]:
countryDimFinal.count()

262

### Step 7 - Write the data to the destination

In [0]:
# Here we are going to write the country dimension to an output csv file
countryDimFinal \
    .coalesce(1) \
    .write.csv('CountryDim', mode='overwrite', header='true')

### Step 8 - Check if the output was written out as expected

In [0]:
!cat CountryDim/*csv | head

country_iso_code,wb_country_code,country_name,country_long_name,region,income_group
AW,ABW,Aruba,Aruba,Latin America & Caribbean,High income
AF,AFG,Afghanistan,Islamic State of Afghanistan,South Asia,Low income
AO,AGO,Angola,People's Republic of Angola,Sub-Saharan Africa,Lower middle income
AL,ALB,Albania,Republic of Albania,Europe & Central Asia,Upper middle income
AD,AND,Andorra,Principality of Andorra,Europe & Central Asia,High income
1A,ARB,Arab World,Arab World,"",""
AE,ARE,United Arab Emirates,United Arab Emirates,Middle East & North Africa,High income
AR,ARG,Argentina,Argentine Republic,"",""
AM,ARM,Armenia,Republic of Armenia,Europe & Central Asia,Lower middle income


## Exercise

#### Transform the Series Dataset and make it available in the dataframe seriesDim
- Filter only for series that have Annual periodicity
- Get the following columns and rename the selected columns to prepare further processing  

| Name in CSV | Column Name |
| ------------- |:-------------:|
| Series Code | indicator_code |
| Indicator Name | indicator_name |
| Periodicity | periodicity |
| Aggregation Method | aggregation_method |


##### How many series do you end up with ?

In [0]:
#Your solution here
#seriesDim = ??? 

#### What data quality checks were you able to perform ?

- Are there any duplicate codes ?
- Are all indicator_codes following the same pattern ?
- Is the case on columns consistent ? 

In [0]:
#Your solution here


### Complex Transformations
##### Problem Statement
We want to measure the cellular and broadband penetration in comparison to the population demographics for every country.  It'll also be helpful to get some insights on annual global aggregates.

###### Our dataset has multiple types of metrics.  The only ones that we care about are simple aggregates.

In [0]:
simpleAggIndicators = seriesDim \
    .filter("lower(aggregation_method) = 'sum'") \
    .select("indicator_code", "indicator_name") \
    .orderBy("indicator_code")

showDF(simpleAggIndicators, limitRows = 500, truncate = False)

NameError: ignored

##### Only keep the indicators that are relevant to requirements i.e. Population indicators and Cellular and Broadband penetration

In [0]:
targetIndicators = simpleAggIndicators \
    .filter("lower(indicator_name) like '%population%total%' " + 
            " or lower(indicator_name) like '%cellular%' " +
            " or lower(indicator_name) like '%broadband%'") \
    .filter("lower(indicator_name) not like '%refugee%'")

showDF(targetIndicators)

##### Now that we have identified the various indicators of interest, we can continue with getting the metrics for these indicators

In [0]:
# Keep the columns that are relevant for further transformations
indicatorsData = indicators \
    .withColumnRenamed("Indicator Code", "indicator_code") \
    .withColumnRenamed("Country Code", "wb_country_code") \
    .drop("Indicator Name") \
    .drop("Country Name") \
    .drop("_c62")

In [0]:
#Keep only the indicators that we care about
targetIndicatorsData = indicatorsData.join(targetIndicators \
                                         , indicatorsData.indicator_code == targetIndicators.indicator_code) \
    .drop(targetIndicators.indicator_code)

In [0]:
showDF(targetIndicatorsData)

#### The output that we see currently isn't the most ideal from a modeling perspective.  
A well-modeled dataset should allow for data to be easily augmented.  E.g. instead of having a column for each year (wide format), we would prefer a row for each year (long format). This allows us to easily add more rows in the future. This wide-to-long transformation is also a common preprocessing step when building models.

We want something similar to the output of the following code block:

In [0]:
indicatorsSample = targetIndicatorsData \
    .select(col("wb_country_code"),
            col("indicator_code"),
            lit("1960").alias("year"),
            col("1960").alias("indicator_value")) \
    .filter("indicator_value >= 0.0")

showDF(indicatorsSample)

##### Let us start by getting the list of years that we have metrics for

In [0]:
yearList = [x for x in targetIndicatorsData.schema.names \
             if x != 'wb_country_code' and x != 'indicator_code' and x != 'indicator_name'] 

print(yearList)

In [0]:
#Cheat for creating a dataframe with no rows 
indicatorsDF = indicatorsSample.filter('1 = 0')

#Iterate through the list of years and store the rows in the DataFrame we created above
for indicatorYear in yearList:
    print("Processing indicators for " + indicatorYear)
    yearIndicatorDF = targetIndicatorsData \
        .select(col("wb_country_code")
                , col("indicator_code")
                , lit(indicatorYear).alias("year")
                , col(indicatorYear).alias("indicator_value")) \
        .filter("indicator_value >= 0")
    indicatorsDF = indicatorsDF.union(yearIndicatorDF)    

#### Let's cache the dataset to iterate over it

In [0]:
# You can iterate over a dataframe that is already computed by caching it once and using it repeatedly
indicatorsDF.cache()

#Force the data to be cached
indicatorsDF.count()

In [0]:
#Check the indicator counts per year
#showDF(indicatorsDF.groupBy('year').agg(count("*")).orderBy("year"), limitRows=100)

#### Getting yearly indicator totals

In [0]:
yearPivot = indicatorsDF.groupBy('year').pivot('indicator_code').sum('indicator_value') 

In [0]:
showDF(yearPivot.orderBy('year'))

In [0]:
yearPivot.printSchema()

In [0]:
yearPivotDF = yearPivot.orderBy('year') \
    .withColumnRenamed('IT.CEL.SETS', 'cellular_subscriptions') \
    .withColumnRenamed('IT.NET.BBND', 'broadband_subscriptions') \
    .withColumnRenamed('SP.POP.0014.TO', 'population_age_0_to_14') \
    .withColumnRenamed('SP.POP.1564.TO', 'population_age_15_64') \
    .withColumnRenamed('SP.POP.65UP.TO', 'population_age_65_and_above') \
    .withColumnRenamed('SP.POP.TOTL', 'population')

#### Data Quality Checkpoint 

In [0]:
# You can iterate over a dataframe that is already computed by caching it onces and using it repeatedly
yearPivotDF.cache()

#Forces the data to be cached
yearPivotDF.count()

In [0]:
yearPivotDF.filter('population_age_0_to_14 < 0').count()

In [0]:
yearPivotDF.filter('population_age_15_64 < 0').count()

In [0]:
yearPivotDF.filter('population_age_0_to_14 < 0').count()

In [0]:
yearPivotDF.filter('population_age_65_and_above < 0').count()

In [0]:
yearPivotDF.filter('population < 0').count()

In [0]:
yearPivotDF.filter('cellular_subscriptions < 0').count()

In [0]:
yearPivotDF.filter('broadband_subscriptions < 0').count()

In [0]:
yearPivotDF.filter('population_age_0_to_14 > population').count()

In [0]:
yearPivotDF.filter('population_age_15_64 > population').count()

In [0]:
yearPivotDF.filter('population_age_65_and_above > population').count()

In [0]:
yearPivotDF.filter('(population_age_0_to_14 + population_age_15_64 + population_age_65_and_above) > population').count()

In [0]:
#Write the yearly totals to a CSV File
yearPivotDF \
    .select(col('year')
            , col('population').cast(DecimalType(38, 2))
            , col('population_age_0_to_14').cast(DecimalType(38, 2))
            , col('population_age_15_64').cast(DecimalType(38, 2))
            , col('population_age_65_and_above').cast(DecimalType(38, 2))
            , col('broadband_subscriptions').cast(DecimalType(38, 2))
            , col('cellular_subscriptions').cast(DecimalType(38, 2))) \
    .coalesce(1) \
    .write.csv('../output/YearlyStats', mode='overwrite', header='true')

#### Getting yearly regional totals

In [0]:
regionalIndicators = indicatorsDF.join(countryDimFinal
                                       , indicatorsDF.wb_country_code == countryDim.wb_country_code
                                       , "inner") \
    .select(countryDim.region
            , indicatorsDF.wb_country_code
            , indicatorsDF.year
            , indicatorsDF.indicator_code
            , indicatorsDF.indicator_value)

In [0]:
showDF(regionalIndicators)

In [0]:
regionalPivot = regionalIndicators.groupBy('region', 'year').pivot('indicator_code').sum('indicator_value')

In [0]:
showDF(regionalPivot.orderBy('region', 'year'), limitRows=100)

In [0]:
#Write the regional-yearly totals to a CSV File
regionalPivot.filter('region is not null') \
    .orderBy('region','year') \
    .withColumnRenamed('IT.CEL.SETS', 'cellular_subscriptions') \
    .withColumnRenamed('IT.NET.BBND', 'broadband_subscriptions') \
    .withColumnRenamed('SP.POP.0014.TO', 'population_age_0_to_14') \
    .withColumnRenamed('SP.POP.1564.TO', 'population_age_15_64') \
    .withColumnRenamed('SP.POP.65UP.TO', 'population_age_65_and_above') \
    .withColumnRenamed('SP.POP.TOTL', 'population') \
    .select(col('region')
            , col('year')
            , col('population').cast(DecimalType(38, 2))
            , col('population_age_0_to_14').cast(DecimalType(38, 2))
            , col('population_age_15_64').cast(DecimalType(38, 2))
            , col('population_age_65_and_above').cast(DecimalType(38, 2))
            , col('broadband_subscriptions').cast(DecimalType(38, 2))
            , col('cellular_subscriptions').cast(DecimalType(38, 2))) \
    .coalesce(1) \
    .write.csv('../output/RegionalStats', mode='overwrite', header='true')

## Short Exercise

Becky finds the regional metrics interesting, but she wants to look at these metrics at a country level for each year.  Can you adapt the regional pivot that we computed earlier to get the metrics for each year and country ?

In [0]:
#Your solution here

## Exercise

Kat wants to identify the countries that are conducive to start a business.  She thinks that it would suffice to look at the most recent metrics for the following:
- Gross National Income (GNI)
- Cost of business start-up procedures
- Number of days required to start a business (male, female, and overall)
- Number of start-up procedures to register a business
- GDP
- GDP per capita
- Business Regulatory Environment
- Ease of doing business index (Only available in 2017)

Write the data to a csv file called 'BusinessStartupData'.

In [0]:
#Hint - Start by matching up indicators that might have descriptions matching what Kat is looking for. E.g.:
showDF(seriesDim.filter("indicator_name = 'GNI'" +
                        " or lower(indicator_name) like '%business%'").orderBy("indicator_code"), limitRows = 500)

In [0]:
# Your code here

# Modeling

Let's use Spark to do some modeling!

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

Let's take the dataset you created in the previous exercise and predict 'Ease of Business' using other indicators.

In [0]:
countryBusinessStartupPivot = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("BusinessStartupData")

In [0]:
#Input all the features in one vector column
features = ["GDP", "GDP Per Capita", "GNI", "Startup Time",
            "Startup Cost", "Startup Cost Pct of GNI", "Startup Procedures", "Business Regulation"]

# Vector Assembler combines raw features and features generated by different feature transformers 
# into the single feature vector needed to train ML models.
assembler = VectorAssembler(inputCols=features, outputCol = 'Attributes')

output = assembler.setHandleInvalid("skip").transform(countryBusinessStartupPivot.filter(col('Ease of Business').isNotNull()))

#'Attributes' are the input features and 'Ease of Business' is the target column.
finalized_data = output.select("Attributes", "Ease of Business")

finalized_data.show()

+--------------------+----------------+
|          Attributes|Ease of Business|
+--------------------+----------------+
|[9.103831278E9,83...|          151.00|
|[1.4342385958E10,...|          179.00|
|[7.91061837E9,729...|          181.00|
|[3.369621899E9,45...|          160.00|
|[2.27748E11,1178....|          147.00|
|[1.6833353304E10,...|          140.00|
|[7.32044136E8,667...|          129.00|
|[1.2424107945E10,...|          180.00|
|[1.627046588E9,35...|          172.00|
|[1.3421822111E10,...|          143.00|
|[6.315715818E9,10...|           77.00|
|[8.458800788E9,96...|          123.00|
|[8.085878853E9,39...|          144.00|
|[7.326645709E9,20...|           44.00|
|[4.062377053E8,37...|           89.00|
|[1.4858345267E10,...|          138.00|
|[1.084315801E9,53...|          146.00|
|[1.790417939E8,33...|          149.00|
|[1.928690844E8,16...|          157.00|
|[3.0547324905E10,...|          182.00|
+--------------------+----------------+
only showing top 20 rows



In [0]:
#Split training and testing data
train_data,test_data = finalized_data.randomSplit([0.8,0.2])

regressor = LinearRegression(featuresCol = 'Attributes', labelCol = 'Ease of Business')

#Learn to fit the model from training set
regressor = regressor.fit(train_data)

#To predict the prices on testing set
pred = regressor.evaluate(test_data)

#Predict the model
pred.predictions.show()

+--------------------+----------------+------------------+
|          Attributes|Ease of Business|        prediction|
+--------------------+----------------+------------------+
|[1.6833353304E10,...|          140.00| 121.2723619115442|
|[8.085878853E9,39...|          144.00|113.02867824748559|
|[1.928690844E8,16...|          157.00|152.27471530136742|
|[7.38041747E8,378...|           87.00| 113.8654191546719|
|[1.035620086E10,4...|          162.00|140.91541281804916|
|[1.9856634053E10,...|          105.00|118.24913879612849|
|[8.867464234E8,14...|          116.00| 140.6478735145592|
|[2.1413614655E10,...|          183.00|146.91456643648525|
|[1.67771E11,1029....|          177.00|147.25354925521165|
|[5.060218541E8,68...|           98.00|100.12359132745794|
|[7.6149479922E10,...|          170.00|148.91975047555985|
|[1.1102149875E10,...|          141.00| 99.22452006961993|
|[2.234759342E9,28...|           75.00|120.16114635444366|
|[1.192299136E9,93...|          178.00|167.7258793093201

In [0]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

summary = pd.DataFrame({'feature': ['Intercept'] + features,
                        'coefficient': [regressor.intercept] + regressor.coefficients.tolist(),
                        'pvalue': regressor.summary.pValues})
summary

,feature,coefficient,pvalue
0,Intercept,235.93695,0.49581
1,GDP,-0.00000,0.65339
2,GDP Per Capita,0.00098,0.62237
3,GNI,0.00000,0.12556
4,Startup Time,-0.37788,0.25113
5,Startup Cost,0.00000,0.39788
6,Startup Cost Pct of GNI,0.10033,0.17010
7,Startup Procedures,2.56238,0.00007
8,Business Regulation,-38.09088,0.00000


In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="Ease of Business", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 29.115
MSE: 847.686
MAE: 25.363
r2: 0.430
